# Offline reinforcement learning with Ray AIR
In this example, we'll train a reinforcement learning agent using offline training.

Offline training means that the data from the environment (and the actions performed by the agent) have been stored on disk. In contrast, online training samples experiences live by interacting with the environment.

Let's start with installing our dependencies:

In [1]:
# !pip install -qU "ray[rllib]" gymnasium

Now we can run some imports:

In [2]:
import argparse
import gymnasium as gym
import os

import numpy as np
import ray
from ray.air import Checkpoint
from ray.air.config import RunConfig
from ray.train.rl.rl_predictor import RLPredictor
from ray.train.rl.rl_trainer import RLTrainer
from ray.air.config import ScalingConfig
from ray.air.result import Result
from ray.rllib.algorithms.bc import BC
from ray.tune.tuner import Tuner

/Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
/Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
/Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:43: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from gen

/Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


We will be training on offline data - this means we have full agent trajectories stored somewhere on disk and want to train on these past experiences.

Usually this data could come from external systems, or a database of historical data. But for this example, we'll generate some offline data ourselves and store it using RLlibs `output_config`.

In [3]:
def generate_offline_data(path: str):
    print(f"Generating offline data for training at {path}")
    trainer = RLTrainer(
        algorithm="PPO",
        run_config=RunConfig(stop={"timesteps_total": 5000}),
        config={
            "env": "CartPole-v1",
            "output": "dataset",
            "output_config": {
                "format": "json",
                "path": path,
                "max_num_samples_per_file": 1,
            },
            "batch_mode": "complete_episodes",
            "framework": "torch"
        },
    )
    trainer.fit()

Here we define the training function. It will create an `RLTrainer` using the `PPO` algorithm and kick off training on the `CartPole-v1` environment. It will use the offline data provided in `path` for this.

In [4]:
def train_rl_bc_offline(path: str, num_workers: int, use_gpu: bool = False) -> Result:
    print("Starting offline training")
    dataset = ray.data.read_json(
        path, parallelism=num_workers, ray_remote_args={"num_cpus": 1}
    )

    trainer = RLTrainer(
        run_config=RunConfig(stop={"training_iteration": 5}),
        scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
        datasets={"train": dataset},
        algorithm=BC,
        config={
            "env": "CartPole-v1",
            "framework": "tf",
            "evaluation_num_workers": 1,
            "evaluation_interval": 1,
            "evaluation_config": {"input": "sampler"},
            "framework": "torch"
        },
    )

    # Todo (krfricke/xwjiang): Enable checkpoint config in RunConfig
    # result = trainer.fit()
    tuner = Tuner(
        trainer,
        _tuner_kwargs={"checkpoint_at_end": True},
    )
    result = tuner.fit()[0]
    return result

Once we trained our RL policy, we want to evaluate it on a fresh environment. For this, we will also define a utility function:

In [5]:
def evaluate_using_checkpoint(checkpoint: Checkpoint, num_episodes) -> list:
    predictor = RLPredictor.from_checkpoint(checkpoint)

    env = gym.make("CartPole-v1")

    rewards = []
    for i in range(num_episodes):
        obs, _ = env.reset()
        reward = 0.0
        terminated = truncated = False
        while not terminated and not truncated:
            action = predictor.predict(np.array([obs]))
            obs, r, terminated, truncated, _ = env.step(action[0])
            reward += r
        rewards.append(reward)

    return rewards

Let's put it all together. First, we initialize Ray and create the offline data:

In [6]:
ray.init(num_cpus=8)

path = "/tmp/out"
generate_offline_data(path)

2023-03-29 16:19:31,577	INFO worker.py:1622 -- Started a local Ray instance.


Generating offline data for training at /tmp/out


(pid=43347) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=43347)   DESCRIPTOR = _descriptor.FileDescriptor(
(pid=43347) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=43347)   _descriptor.FieldDescriptor(
(pid=43347) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:43: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors i

(pid=43347) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=43347)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=43347) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=43347)   distutils.version.LooseVersion(required_tensorflow_version)):
(pid=43347) WARNING:tensorflow:From /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/python/util/deprecation.py:561: calling function (from tensorflow.python.eager.polymorphic_function.polymorphic_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=43347) Instructions for updating:
(pid=43347) experimental_relax_shapes is deprecated, use reduce_retracing i

(pid=43371)   *- Repartition 2:   0%|                                              | 0/1 [00:00<?, ?it/s]


(pid=43371) Write: 0 active, 0 queued, 0.0 MiB objects 3:   0%|                    | 0/1 [00:00<?, ?it/s]



                                                                                                         :00<?, ?it/s]
                                                                                                      

                                                                                                   


                                                                                                



                                                                                             (pid=43371)   if (distutils.version.LooseVersion(tf.__version__) < [repeated 2x across cluster]
(pid=43371) Resource usage vs limits: 0.0/8.0 CPU, 0.0/0.0 GPU, 0.0 MiB/512.0 MiB object_store_memory 0: 
(pid=43371) Repartition: 0 active, 0 queued, 0.0 MiB objects, 0

(pid=43372)   *- Repartition 2: 100%|██████████████████████████████████████| 1/1 [00:00<00:00,  1.07it/s]





(pid=43372) Repartition: 0 active, 0 queued, 0.0 MiB objects, 1 output 1:   0%|    | 0/1 [00:00<?, ?it/s]





(pid=43372) Repartition: 0 active, 0 queued, 0.0 MiB objects, 1 output 1: 100%|█| 1/1 [00:00<00:00,  1.07




(pid=43372) Resource usage vs limits: 0.0/8.0 CPU, 0.0/0.0 GPU, 0.32 MiB/512.0 MiB object_store_memory 0:





(pid=43372) Repartition: 0 active, 0 queued, 0.0 MiB objects, 0 output 1: 100%|█| 1/1 [00:00<00:00,  1.07







(pid=43372) Write: 1 active, 0 queued, 0.32 MiB objects 3:   0%|                   | 0/1 [00:00<?, ?it/s]







(pid=43372) Write: 1 active, 0 queued, 0.32 MiB objects 3: 100%|███████████| 1/1 [00:00<00:00,  1.07it/s]




(pid=43372) Resource usage vs limits: 0.0/8.0 CPU, 0.0/0.0 GPU, 0.0 MiB/512.0 MiB object_store_memory 0: 








(pid=43372) output: 1 queued 4:   0%|                                              | 0/1 [00:00<?, ?it/s]



Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
AIRPPO_293aa_00000,8192,"{'ObsPreprocessorConnector_ms': 0.0019402503967285156, 'StateBufferConnector_ms': 0.0017902851104736328, 'ViewRequirementAgentConnector_ms': 0.03853440284729004}","{'num_env_steps_sampled': 8192, 'num_env_steps_trained': 8192, 'num_agent_steps_sampled': 8192, 'num_agent_steps_trained': 8192}",{},2023-03-29_16-19-49,True,43.48,{},122,43.48,11,94,276,avnishs-mbp-3.lan,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.9658484635719409, 'cur_kl_coeff': 0.29999999999999993, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 8.883838643630346, 'policy_loss': -0.02818693928614569, 'vf_loss': 8.906471814711889, 'vf_explained_var': 0.036158579774200915, 'kl': 0.018512499035662763, 'entropy': 0.6128480241944393, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 1410.5, 'diff_num_grad_updates_vs_sampler_policy': 479.5}}, 'num_env_steps_sampled': 8192, 'num_env_steps_trained': 8192, 'num_agent_steps_sampled': 8192, 'num_agent_steps_trained': 8192}",2,127.0.0.1,8192,8192,8192,4158,8192,4158,0,2,0,0,4158,"{'cpu_util_percent': 22.98333333333333, 'ram_util_percent': 56.98333333333333}",43347,{},{},{},"{'mean_raw_obs_processing_ms': 0.1516845704733748, 'mean_inference_ms': 0.30370324348636124, 'mean_action_processing_ms': 0.045066267048839564, 'mean_env_wait_ms': 0.021038728812942817, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 122.0, 'episode_reward_min': 11.0, 'episode_reward_mean': 43.48, 'episode_len_mean': 43.48, 'episode_media': {}, 'episodes_this_iter': 94, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [75.0, 12.0, 18.0, 14.0, 20.0, 51.0, 75.0, 19.0, 26.0, 36.0, 29.0, 46.0, 11.0, 32.0, 80.0, 31.0, 102.0, 65.0, 13.0, 20.0, 82.0, 33.0, 30.0, 20.0, 56.0, 25.0, 61.0, 17.0, 18.0, 61.0, 12.0, 32.0, 74.0, 71.0, 49.0, 80.0, 36.0, 24.0, 14.0, 19.0, 19.0, 37.0, 59.0, 14.0, 20.0, 53.0, 33.0, 60.0, 54.0, 48.0, 33.0, 65.0, 15.0, 40.0, 29.0, 90.0, 68.0, 24.0, 77.0, 44.0, 36.0, 32.0, 87.0, 37.0, 31.0, 12.0, 86.0, 53.0, 12.0, 56.0, 16.0, 54.0, 47.0, 19.0, 71.0, 19.0, 53.0, 21.0, 23.0, 112.0, 30.0, 14.0, 57.0, 50.0, 25.0, 18.0, 49.0, 79.0, 25.0, 62.0, 22.0, 63.0, 40.0, 122.0, 23.0, 38.0, 18.0, 56.0, 101.0, 108.0], 'episode_lengths': [75, 12, 18, 14, 20, 51, 75, 19, 26, 36, 29, 46, 11, 32, 80, 31, 102, 65, 13, 20, 82, 33, 30, 20, 56, 25, 61, 17, 18, 61, 12, 32, 74, 71, 49, 80, 36, 24, 14, 19, 19, 37, 59, 14, 20, 53, 33, 60, 54, 48, 33, 65, 15, 40, 29, 90, 68, 24, 77, 44, 36, 32, 87, 37, 31, 12, 86, 53, 12, 56, 16, 54, 47, 19, 71, 19, 53, 21, 23, 112, 30, 14, 57, 50, 25, 18, 49, 79, 25, 62, 22, 63, 40, 122, 23, 38, 18, 56, 101, 108]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.1516845704733748, 'mean_inference_ms': 0.30370324348636124, 'mean_action_processing_ms': 0.045066267048839564, 'mean_env_wait_ms': 0.021038728812942817, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.0019402503967285156, 'StateBufferConnector_ms': 0.0017902851104736328, 'ViewRequirementAgentConnector_ms': 0.03853440284729004}}",8.59409,3.74682,8.59409,"{'training_iteratio

(pid=43372) Repartition 1:   0%|                                                   | 0/1 [00:00<?, ?it/s]

(pid=43372) Repartition 2:   0%|                                                   | 0/1 [00:00<?, ?it/s]

(pid=43372)   *- Repartition 2:   0%|                                              | 0/1 [00:00<?, ?it/s]


(pid=43372) Write 3:   0%|                                                         | 0/1 [00:00<?, ?it/s]



(pid=43372) Resource usage vs limits: 0.0/8.0 CPU, 0.0/0.0 GPU, 0.0 MiB/512.0 MiB object_store_memory 0: 



(pid=43372) output: 0 queued 4:   0%|                                              | 0/1 [00:00<?, ?it/s]
(pid=43372) Repartition: 0 active, 0 queued, 0.0 MiB objects, 0 output 1:   0%|    | 0/1 [00:00<?, ?it/s]


(pid=43372) Write: 0 active, 0 queued, 0.0 MiB objects 3:   0%|                    | 0/1 [00:00<?, ?it/s]
(pid=43372) Resource usage vs limits: 0.0/8.0 CPU, 0.0/0.0 GPU, 0.33 MiB/512.0 MiB object_store_memory 0:
(pid=43372) Repartition: 0 active,

Then, we run training:

In [7]:
result = train_rl_bc_offline(path=path, num_workers=2, use_gpu=False)

Starting offline training


(pid=43427) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=43427)   DESCRIPTOR = _descriptor.FileDescriptor(
(pid=43427) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=43427)   _descriptor.FieldDescriptor(
(pid=43427) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:43: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors i

(pid=43427) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=43427)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=43427) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=43427)   distutils.version.LooseVersion(required_tensorflow_version)):
(pid=43427) WARNING:tensorflow:From /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/python/util/deprecation.py:561: calling function (from tensorflow.python.eager.polymorphic_function.polymorphic_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=43427) Instructions for updating:
(pid=43427) experimental_relax_shapes is deprecated, use reduce_retracing i

(RolloutWorker pid=43441) DatasetReader 1 has 2, samples.


(pid=43441) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead. [repeated 4x across cluster]
(pid=43441)   if (distutils.version.LooseVersion(tf.__version__) < [repeated 2x across cluster]
(pid=43441)   distutils.version.LooseVersion(required_tensorflow_version)): [repeated 2x across cluster]
(pid=43441) WARNING:tensorflow:From /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/python/util/deprecation.py:561: calling function (from tensorflow.python.eager.polymorphic_function.polymorphic_function) with experimental_relax_shapes is deprecated and will be removed in a future version. [repeated 2x across cluster]
(pid=43441) Instructions for updating: [repeated 2x across cluster]
(pid=43441) experimental_relax_shapes is deprecated, use reduce_retracing instead [repeated 2x across cluster]
(AIRBC pid=43427) 2023-03-29 16:

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,evaluation,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
AIRBC_3458f_00000,20475,{},"{'num_env_steps_sampled': 20475, 'num_env_steps_trained': 20475, 'num_agent_steps_sampled': 20475, 'num_agent_steps_trained': 20475}",{},2023-03-29_16-20-04,True,nan,{},nan,nan,nan,0,0,"{'episode_reward_max': 43.0, 'episode_reward_min': 9.0, 'episode_reward_mean': 22.3, 'episode_len_mean': 22.3, 'episode_media': {}, 'episodes_this_iter': 10, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [28.0, 14.0, 21.0, 17.0, 9.0, 21.0, 18.0, 31.0, 43.0, 21.0], 'episode_lengths': [28, 14, 21, 17, 9, 21, 18, 31, 43, 21]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.13089791863365513, 'mean_inference_ms': 0.25640968727854496, 'mean_action_processing_ms': 0.04665851593017578, 'mean_env_wait_ms': 0.02178027566555327, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.0021219253540039062, 'StateBufferConnector_ms': 0.0020122528076171875, 'ViewRequirementAgentConnector_ms': 0.039920806884765625}, 'num_agent_steps_sampled_this_iter': 223, 'num_env_steps_sampled_this_iter': 223, 'timesteps_this_iter': 223, 'num_healthy_workers': 1, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}",avnishs-mbp-3.lan,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.38756194710731506, 'policy_loss': 0.6908425092697144, 'total_loss': 0.6908425092697144}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 2000.0, 'num_grad_updates_lifetime': 9.5, 'diff_num_grad_updates_vs_sampler_policy': 8.5}}, 'num_env_steps_sampled': 20475, 'num_env_steps_trained': 20475, 'num_agent_steps_sampled': 20475, 'num_agent_steps_trained': 20475}",5,127.0.0.1,20475,20475,20475,4091,20475,4091,0,2,0,0,4091,{},43427,{},{},{},{},"{'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0, 'connector_metrics': {}}",0.914515,0.163303,0.914515,"{'training_iteration_time_ms': 62.061, 'sample_time_ms': 36.239, 'load_time_ms': 0.341, 'load_throughput': 11992511.437, 'learn_time_ms': 22.095, 'learn_throughput': 185336.343, 'synch_weights_time_ms': 3.322}",1680132004,20475,5,3458f_00000


(pid=43461) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/attr_value_pb2.py:21: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool. [repeated 5x across cluster]
(pid=43461)   DESCRIPTOR = _descriptor.FileDescriptor( [repeated 5x across cluster]
(pid=43461) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/attr_value_pb2.py:82: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool. [repeated 40x across cluster]
(pid=43461)   _descriptor.FieldDescriptor( [repeated 40x across cluster]
(pid=43461) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_pb2.py:176: 

And then, using the obtained checkpoint, we evaluate the policy on a fresh environment:

In [8]:
num_eval_episodes = 3

rewards = evaluate_using_checkpoint(result.checkpoint, num_episodes=num_eval_episodes)
print(f"Average reward over {num_eval_episodes} episodes: " f"{np.mean(rewards)}")

2023-03-29 16:20:05,415	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory /var/folders/jr/6lgb7_ln64v1kppw9szl17rc0000gn/T/tmpk6yck3hw! Trying to extract checkpoint info from other files found in that dir.
2023-03-29 16:20:05,425	INFO policy.py:1285 -- Policy (worker=local) running on CPU.
2023-03-29 16:20:05,425	INFO torch_policy_v2.py:110 -- Found 0 visible cuda devices.


Average reward over 3 episodes: 32.0
